# 决策树算法

## 计算给定数据集的香农熵

In [1]:
from math import log

In [2]:
def calcShannonEnt(dataSet):
    # 得到整个数据集的总数
    numEntries = len(dataSet)
    # 为所有可能分类创建字典
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        labelCounts[currentLabel] = labelCounts.get(currentLabel, 0) + 1
    shannonEnt = 0.0
    for key in labelCounts:
        # 分类的概率
        prob = float(labelCounts[key]) / numEntries
        # 以2为底求对数
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [3]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [4]:
myDat, labels = createDataSet()

In [5]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [6]:
calcShannonEnt(myDat)

0.9709505944546686

## 在原始数据中修改，新增第三个名为maybe的分类，将第一条数据的分类修改为maybe

In [7]:
myDat[0][-1] = 'maybe'

In [8]:
myDat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [9]:
calcShannonEnt(myDat)

1.3709505944546687

## 按照给定特征划分数据集

In [10]:
# dataSet为待划分的数据集，axis为划分数据集的特征值所在的位置， value需要返回的特征的值
# list.extend() 函数用于在列表末尾一次性追加另一个序列中的多个值（用新列表扩展原来的列表）。
# list.append() 方法用于在列表末尾添加新的对象。
def splitDataSet(dataSet, axis, value):
    # 创建新的list对象
    retDataSet = []
    for featVec in dataSet:
        # 发现符合要求的值，则将其添加到新创建的列表中。
        if featVec[axis] == value:
            # 将符合特征的数据抽取出来
            # 将其他的列数据抽取出来
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis + 1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [11]:
myDat, labels = createDataSet()

In [12]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [13]:
splitDataSet(myDat, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [14]:
splitDataSet(myDat, 0, 0)

[[1, 'no'], [1, 'no']]

## 选择最好的数据集划分方式

In [15]:
# 选取特征，划分数据集，计算得出最好的划分数据集的特征
def chooseBestFeatureToSplit(dataSet):
    # 特征值的个数，在本例中为2个
    numFeatures = len(dataSet[0]) - 1
    # 得到信息熵
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):
        # 创建唯一的分类标签列表
        # 重新创建特征值列表
        featList = [example[i] for example in dataSet]
        # 进行去重处理
        uniqueVals = set(featList)
        newEntropy = 0.0
        # 计算每种划分方式的信息熵
        for value in uniqueVals:
            # 按照给定的特征划分数据集
            subDataSet = splitDataSet(dataSet, i, value)
            # 计算概率
            prob = len(subDataSet) / float(len(dataSet))
            # 对所有唯一特征值得到的熵求和
            newEntropy += prob * calcShannonEnt(subDataSet)
        # 取熵最大的特征值
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    # 返回最好特征值划分的索引值
    return bestFeature

In [16]:
myDat, labels = createDataSet()

In [17]:
chooseBestFeatureToSplit(myDat)

0

In [18]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

## 递归构建决策树

得到原始数据集，然后基于最好的属性值划分数据集，由于特征值可能多于两个，因此可能存在大于两个分支的数据集划分。第一次划分之后，数据将被向下传递到树分支的下一个节点，在这个节
点上，我们可以再次划分数据。因此我们可以采用递归的原则处理数据集。

In [19]:
import operator

### 采用多数表决的方法决定叶子节点的分类

In [20]:
# classList为分类名称的列表
def majorityCnt(classList):
    # 创建键值为classList中唯一值的数据字典，字典对象存储了classList中每个类标签出现的频率
    classCount = {}
    for vote in classList:
        classCount[vote] = classCount.get(vote, 0) + 1
    sortedClassCount = sorted(classCount.items(), 
                             key=operator.itemgetter(1),
                             reverse=True)
    return sortedClassCount[0][0]

### 创建树

In [21]:
# dataSet为数据集，labels为标签列表
def createTree(dataSet, labels):
    # 得到类标签
    classList = [example[-1] for example in  dataSet]
    # 类别完全相同则停止继续划分
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 遍历完所有特征则返回出现次数最多的
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    # 选择最好的数据集划分的特征值
    bestFeat = chooseBestFeatureToSplit(dataSet)
    # 获得特征值的标签
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    # 删除处理过的标签
    del labels[bestFeat]
    # 取出同一个特征值的数据
    featValues = [example[bestFeat] for example in dataSet]
    # 过滤重复数据
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

In [22]:
myDat, labels = createDataSet()

In [23]:
myTree = createTree(myDat, labels)

In [24]:
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

## 使用决策树的分类函数

In [25]:
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    # 将标签字符串转换为索引
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featLabels, testVec)
            else:
                # 到达叶子节点，返回当前节点的分类标签
                classLabel = secondDict[key]
    return classLabel

In [26]:
myDat, labels = createDataSet()

In [27]:
labels

['no surfacing', 'flippers']

In [29]:
def retrieveTree(i):
    listOfTrees = [{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}},
                   {'no surfacing': {0: 'no', 1: {'flippers': {0: {'head': {0: 'no', 1: 'yes'}}, 1: 'no'}}}}
                  ]
    return listOfTrees[i]

In [30]:
myTree = retrieveTree(0)

In [31]:
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

### 如果surfacing,no flippers，得出结论不是鱼

In [33]:
classify(myTree, labels, [1, 0])

'no'

### 如果surfacing,flippers，得出结论是鱼

In [35]:
classify(myTree, labels, [1, 1])

'yes'

## 使用算法：决策树的存储

In [38]:
def storeTree(inputTree, fileName):
    import pickle
    fw = open(fileName, 'wb')
    pickle.dump(inputTree, fw)
    fw.close()
    
def grabTree(fileName):
    import pickle
    fr = open(fileName, 'rb')
    return pickle.load(fr)

In [39]:
storeTree(myTree, 'classifierStorage.txt')

In [40]:
grabTree('classifierStorage.txt')

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}